############# Markdown parser in Jupyter notebooks ##################

<div class="alert alert-block alert-info"> <b>NOTE</b> Use blue boxes for Tips and notes. </div>

<div class="alert alert-block alert-success"> Use green boxes sparingly, and only for some specific purpose that the other boxes can't cover. For example, if you have a lot of related content to link to, maybe you decide to use green boxes for related links from each section of a notebook. </div>

<div class="alert alert-block alert-warning"> Use yellow boxes for examples that are not inside code cells, or use for mathematical formulas if needed. </div>

<div class="alert alert-block alert-danger"> In general, just avoid the red boxes. </div>

# MPI Introduction

An introduction to basic concept of **Message Passing Interface** (MPI)


<div class="alert alert-block alert-warning"> The MPI is a <b>standard</b> decided by <a href="https://www.mpi-forum.org/">MPI Forum</a>. </div>

Born in 1992, first version released in 1994.
Versions:
* *MPI-1*, 1994
* *MPI-2*, 1996
* *MPI-3*, 2005
* *MPI-4*, 2022

<div class="alert alert-block alert-danger">MPI for C++ is deprecated from version MPI 3.0</div>

Different languages supported:
* C/C++
* Fortran
* Pyhton

<div class="alert alert-block alert-warning"> Different <b>implementation</b> of the standard. </div>

Both **open source** and **proprietary**:
* *open-mpi*: https://www.open-mpi.org/
* *mpich*: https://www.mpich.org/
* *intelmpi* (proprietary): https://www.intel.com/content/www/us/en/developer/tools/oneapi/mpi-library.html
* *mpi4py*: https://mpi4py.readthedocs.io/en/stable/index.html

<div class="alert alert-block alert-info"> <b>DOCUMENTATION</b>: we are going to use C/C++ <a href="https://www.open-mpi.org/doc/v4.1/">https://www.open-mpi.org/doc/v4.1/</a>.</div>

<div class="alert alert-block alert-success"> Be aware of which <b>compiler</b> you are using - for some implementations (e.g. openmpi, intelmpi) multiple compilers are available. </div>

## MPI Init and Finalize

Calls used to initialize and terminate the parallel session.

* `MPI_Init`: see https://www.open-mpi.org/doc/v4.1/man3/MPI_Init.3.php
* `MPI_Finalize`: see https://www.open-mpi.org/doc/v4.1/man3/MPI_Finalize.3.php

The following program initialize a parallel session.

### C++

In [ ]:
%%writefile main_init.cpp

#include <iostream>
#include <mpi.h>

int main(int argc, char **argv) 
{
    // Initialize MPI
    // This must always be called before any other MPI functions
    MPI_Init(&argc, &argv);
    
    std::cout<< "Hello world!"<< std::endl;

    // Finalize MPI
    // This must always be called after all other MPI functions
    MPI_Finalize();

    return 0;
}

<div class="alert alert-block alert-info"> The arguments in <code>MPI_Init</code> are <strong>not used</strong> anymore but some compilers insist they are there. </div>

In [ ]:
%%sh

# compile program
mkdir -p ./debug_init
cd debug_init
cmake -DSOURCES="main_init.cpp" ..
make

In [ ]:
%%sh

# run program
cd debug_init
mpirun -np 4 program

### Python

In [ ]:
%%writefile main_init.py

import mpi4py
mpi4py.rc.initialize = False  # do not initialize MPI automatically
mpi4py.rc.finalize = False    # do not finalize MPI automatically

from mpi4py import MPI # import the 'MPI' module

# manual initialization of the MPI environment
MPI.Init()

print("Hello world!")

# manual finalization of the MPI environment
MPI.Finalize()

In [ ]:
%%sh

# run program
mpirun -np 4 python main_init.py

## MPI Communicators

It is possible to divide the total number of tasks into groups called **communicators**.
The variable identifying a communicator identifies those tasks which can communicate with each other.

<div class="alert alert-block alert-info">The default communicator is called <code>MPI_COMM_WORLD</code> and includes <b>all</b> the tasks available to the program.</div>

<img src="./Images/COMM_WORLD.png" width=20% style="margin-left:auto; margin-right:auto">


* `MPI_Comm_size`: see https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_size.3.php
* `MPI_Comm_rank`: see https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_rank.3.php

In [ ]:
%%writefile main_communicators.cpp

#include <iostream>
#include <mpi.h>

int main(int argc, char **argv)
{
    int err;
    err = MPI_Init(&argc, &argv);
    
    int nprocs, my_rank;
    
    // Get the number of processes in MPI_COMM_WORLD
    err = MPI_Comm_size(MPI_COMM_WORLD, &nprocs);
    
    // Get the rank of this process in MPI_COMM_WORLD
    err = MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);

    std::cout<< "Hello I am process "<< my_rank<< " of "<< nprocs<< " processes"<< std::endl; 
    
    err = MPI_Finalize();
    
    return 0;
}

<div class="alert alert-block alert-success">
    <ol>
        <li> Remember that <b>every</b> process is running the same code independently 
        <li> At the end of the call, rank will have a <b>different</b> value for every process!
    </ol>
</div>

In [ ]:
%%sh

# compile program
mkdir -p ./debug_communicators
cd debug_communicators
cmake -DSOURCES="main_communicators.cpp" ..
make

In [ ]:
%%sh

# run program
cd debug_communicators
mpirun -np 4 program

Communicators can be created with different MPI routines:
* `MPI_Comm_split`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_split.3.php
* `MPI_Comm_dup`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_dup.3.php
* `MPI_Comm_create`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_create.3.php
* ...

<div class="alert alert-block alert-danger"> <b>NOTE</b>: Once a new communicator is created we shall destroy it before closing the application:</div>

* `MPI_Comm_free`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_free.3.php

<div class="alert alert-block alert-info"> We are going to see more about communicators when we are going to see <b>Topologies</b>. </div>


### Example of MPI Comm Split

`MPI_Comm_split`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_split.3.php
<img src="./Images/comm_split.png" width=40% style="margin-left:auto; margin-right:auto">

In [ ]:
%%writefile main_comm_split.cpp

#include <iostream>
#include <mpi.h>
int main(int argc, char **argv)
{
    int err;
    MPI_Init(&argc, &argv);
    
    int nprocs, my_rank;
    
    // Get the rank and size in the original communicator
    int world_rank, world_size;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);

    unsigned int color = world_rank / 4; // Determine color based on row
    
    // Split the communicator based on the color and use the
    // original rank for ordering
    MPI_Comm row_comm;
    err = MPI_Comm_split(MPI_COMM_WORLD, color, world_rank, &row_comm);
    
    int row_rank, row_size;
    MPI_Comm_rank(row_comm, &row_rank);
    MPI_Comm_size(row_comm, &row_size);
    
    std::cout<< "WORLD RANK/SIZE: "<< world_rank<< "/"<< world_size<< "\t";
    std::cout<< "ROW RANK/SIZE: "<< row_rank<< "/"<< row_size<< std::endl;
    
    // Free the communicator
    MPI_Comm_free(&row_comm);
    
    MPI_Finalize();
    
    return 0;
}

In [ ]:
%%sh

# compile program
mkdir -p ./debug_comm_split
cd debug_comm_split
cmake -DSOURCES="main_comm_split.cpp" ..
make

In [ ]:
%%sh

# run program
cd debug_comm_split
mpirun --oversubscribe -np 16 program